<a href="https://colab.research.google.com/github/monica-millay/capstone-data-cleaning/blob/main/Capstone_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone project data cleaning

Welcome! This is the Python notebook where I will create the datasets for our capstone project *Right to Rent: Eviction Filings and Right-to-Work Law*.




##Install packages

In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
#install packages
import pandas as pd
import requests
print("Success!")

Success!


# Create sample

##Court-issued

###Upload Eviction Lab data
Source: https://data-downloads.evictionlab.org/#data-for-analysis/

File: county_court-issued_2000_2018.csv

In [3]:
link = 'https://drive.google.com/file/d/1Yj_-TJCM0Ze3ZHCK4O4z6ooQbG43nDj2/view'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('county_court-issued_2000_2018.csv')

court_raw_data = pd.read_csv('county_court-issued_2000_2018.csv')
print('Success!')

Success!


In [4]:
court_raw_data["filing_rate"] = (court_raw_data["filings_observed"]/court_raw_data["renting_hh"])*100
court_raw_data["filing_rate"] = court_raw_data["filing_rate"].round(decimals=2)
court_raw_data["threatened_rate"] = (court_raw_data["hh_threat_observed"]/court_raw_data["renting_hh"])*100
court_raw_data["threatened_rate"] = court_raw_data["threatened_rate"].round(decimals=2)
court_raw_data.head()

,state,county,fips_state,fips_county,year,renting_hh,filings_observed,ind_filings_court_issued_lt,hh_threat_observed,filing_rate,threatened_rate
0,Alabama,Autauga County,1,1001,2000,3074,109,0,106.0,3.55,3.45
1,Alabama,Autauga County,1,1001,2001,3264,75,0,NaN,2.30,NaN
2,Alabama,Autauga County,1,1001,2002,3454,94,0,NaN,2.72,NaN
3,Alabama,Autauga County,1,1001,2003,3644,100,0,NaN,2.74,NaN
4,Alabama,Autauga County,1,1001,2004,3834,112,0,NaN,2.92,NaN


In [5]:
court_raw_data = court_raw_data.rename(columns={"fips_county":"cofips","filings_observed":"filings","hh_threat_observed": "threatened"})

###Upload RTW data & merge

In [6]:
link2 = 'https://drive.google.com/file/d/1bCU082dOs9Xf-76eM_5VF04cJ03UBp3M/view'

# to get the id part of the file
id = link2.split("/")[-2]

downloaded2 = drive.CreateFile({'id':id})
downloaded2.GetContentFile('All RTW states.csv')

RTW_states = pd.read_csv('All RTW states.csv')

In [7]:
#add year that RTW law was passed to eviction dataset

court_merged_data = court_raw_data.merge(RTW_states, how="left", on="state")

In [8]:
#assign data to treatment and comparison groups

def RTW_year(row):
    year = row["year RTW adopted"]

    if year < 2000:
        return "before 2000"
    elif year >= 2000:
        return "treatment"
    else:
        return "comparison"

court_merged_data["group"] = court_merged_data.apply(RTW_year, axis=1)

In [9]:
#create pre and post dummy

import numpy as np

year = court_merged_data.loc[:,("year")]
state = court_merged_data.loc[:,("state")]

conditions = [
    (state == 'Indiana') & (year > 2012),
    (state == 'Kentucky') & (year > 2017),
    (state == 'Michigan') & (year > 2012),
    (state == 'Oklahoma') & (year > 2001),
    (state == 'Wisconsin') & (year > 2015),
]


choices = [1, 1, 1, 1, 1]

court_merged_data['post treatment'] = np.select(conditions, choices, default=0)
court_merged_data.head()


,state,county,fips_state,cofips,year,renting_hh,filings,ind_filings_court_issued_lt,threatened,filing_rate,threatened_rate,year RTW adopted,group,post treatment
0,Alabama,Autauga County,1,1001,2000,3074,109,0,106.0,3.55,3.45,1953.0,before 2000,0
1,Alabama,Autauga County,1,1001,2001,3264,75,0,NaN,2.30,NaN,1953.0,before 2000,0
2,Alabama,Autauga County,1,1001,2002,3454,94,0,NaN,2.72,NaN,1953.0,before 2000,0
3,Alabama,Autauga County,1,1001,2003,3644,100,0,NaN,2.74,NaN,1953.0,before 2000,0
4,Alabama,Autauga County,1,1001,2004,3834,112,0,NaN,2.92,NaN,1953.0,before 2000,0


###Include only treatment and comparison states

In [10]:
court_sample = court_merged_data[
    (court_merged_data['state'] == 'Indiana') | (court_merged_data['state'] == 'Illinois') |
    (court_merged_data['state'] == 'Michigan') | (court_merged_data['state'] == 'Ohio') |
    (court_merged_data['state'] == 'Oklahoma') | (court_merged_data['state'] == 'Missouri') |
    (court_merged_data['state'] == 'Wisconsin') | (court_merged_data['state'] == 'Minnesota')]
court_sample

,state,county,fips_state,cofips,year,renting_hh,filings,ind_filings_court_issued_lt,threatened,filing_rate,threatened_rate,year RTW adopted,group,post treatment
7350,Illinois,Cook County,17,17031,2001,830604,37452,0,NaN,4.51,NaN,NaN,comparison,0
7351,Illinois,Cook County,17,17031,2002,829703,35331,0,NaN,4.26,NaN,NaN,comparison,0
7352,Illinois,Cook County,17,17031,2003,828803,32975,0,NaN,3.98,NaN,NaN,comparison,0
7353,Illinois,Cook County,17,17031,2004,827902,31937,0,NaN,3.86,NaN,NaN,comparison,0
7354,Illinois,Cook County,17,17031,2005,827002,30278,0,NaN,3.66,NaN,NaN,comparison,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33150,Wisconsin,Wood County,55,55141,2013,8820,204,0,196.0,2.31,2.22,2015.0,treatment,0
33151,Wisconsin,Wood County,55,55141,2014,8936,281,0,262.0,3.14,2.93,2015.0,treatment,0
33152,Wisconsin,Wood County,55,55141,2015,9053,261,0,246.0,2.88,2.72,2015.0,treatment,0
33153,Wisconsin,Wood County,55,55141,2016,9170,316,0,295.0,3.45,3.22,2015.0,treatment,1


###Add unique identifier cofips_year

In [11]:
#create unique identifier to use when merging Census data

def cofips_year(row):
    year = row["year"]
    cofips = row["cofips"]
    if year < 2005:
        return str(cofips)+"_2000"
    elif (year > 2004) & (year < 2010):
        return str(cofips)+"_2009"
    elif (year > 2009) & (year < 2015):
        return str(cofips)+"_2014"
    elif (year > 2014):
        return str(cofips)+"_2018"
    else:
        return "invalid"

In [12]:
court_sample["cofips_year"] = court_sample.apply(cofips_year, axis=1)
court_sample

<ipython-input-12-2aefdf5ff9f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  court_sample["cofips_year"] = court_sample.apply(cofips_year, axis=1)


,state,county,fips_state,cofips,year,renting_hh,filings,ind_filings_court_issued_lt,threatened,filing_rate,threatened_rate,year RTW adopted,group,post treatment,cofips_year
7350,Illinois,Cook County,17,17031,2001,830604,37452,0,NaN,4.51,NaN,NaN,comparison,0,17031_2000
7351,Illinois,Cook County,17,17031,2002,829703,35331,0,NaN,4.26,NaN,NaN,comparison,0,17031_2000
7352,Illinois,Cook County,17,17031,2003,828803,32975,0,NaN,3.98,NaN,NaN,comparison,0,17031_2000
7353,Illinois,Cook County,17,17031,2004,827902,31937,0,NaN,3.86,NaN,NaN,comparison,0,17031_2000
7354,Illinois,Cook County,17,17031,2005,827002,30278,0,NaN,3.66,NaN,NaN,comparison,0,17031_2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33150,Wisconsin,Wood County,55,55141,2013,8820,204,0,196.0,2.31,2.22,2015.0,treatment,0,55141_2014
33151,Wisconsin,Wood County,55,55141,2014,8936,281,0,262.0,3.14,2.93,2015.0,treatment,0,55141_2014
33152,Wisconsin,Wood County,55,55141,2015,9053,261,0,246.0,2.88,2.72,2015.0,treatment,0,55141_2018
33153,Wisconsin,Wood County,55,55141,2016,9170,316,0,295.0,3.45,3.22,2015.0,treatment,1,55141_2018


##Proprietary

###Upload Eviction Lab data

Source: https://data-downloads.evictionlab.org/#data-for-analysis/

File: county_proprietary_valid_2000_2018.csv

In [13]:
link3 = 'https://drive.google.com/file/d/1SEJRfpELN9SKESjRyZZXIErDS5MiCKvi/view'

import pandas as pd

# to get the id part of the file
id = link3.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('county_proprietary_valid_2000_2018.csv')

prop_raw_data = pd.read_csv('county_proprietary_valid_2000_2018.csv')
print('Success!')

Success!


###Merge with RTW data

In [14]:
prop_merged_data = prop_raw_data.merge(RTW_states, how="left", on="state")

In [15]:
#assign data to treatment and comparison groups

def RTW_year(row):
    year = row["year RTW adopted"]

    if year < 2000:
        return "before 2000"
    elif year >= 2000:
        return "treatment"
    else:
        return "comparison"

prop_merged_data["group"] = prop_merged_data.apply(RTW_year, axis=1)

In [16]:
import numpy as np

year = prop_merged_data.loc[:,("year")]
state = prop_merged_data.loc[:,("state")]

conditions = [
    (state == 'Indiana') & (year > 2012),
    (state == 'Kentucky') & (year > 2017),
    (state == 'Michigan') & (year > 2012),
    (state == 'Oklahoma') & (year > 2001),
    (state == 'Wisconsin') & (year > 2015),
]


choices = [1, 1, 1, 1, 1]

prop_merged_data['post treatment'] = np.select(conditions, choices, default=0)
prop_merged_data.head()


,cofips,county,state,year,type,filings,filing_rate,threatened,threatened_rate,judgements,judgement_rate,year RTW adopted,group,post treatment
0,1001,Autauga County,Alabama,2001,observed,84,2.73,83,2.70,NaN,NaN,1953.0,before 2000,0
1,1001,Autauga County,Alabama,2002,observed,100,3.25,97,3.16,NaN,NaN,1953.0,before 2000,0
2,1001,Autauga County,Alabama,2003,observed,102,3.32,90,2.93,NaN,NaN,1953.0,before 2000,0
3,1001,Autauga County,Alabama,2008,observed,130,3.17,126,3.07,NaN,NaN,1953.0,before 2000,0
4,1001,Autauga County,Alabama,2009,observed,108,2.63,104,2.53,NaN,NaN,1953.0,before 2000,0


###Include only treatment and comparison states

In [17]:
prop_sample = prop_merged_data[
    (prop_merged_data['state'] == 'Indiana') | (prop_merged_data['state'] == 'Illinois') |
    (prop_merged_data['state'] == 'Michigan') | (prop_merged_data['state'] == 'Ohio') |
    (prop_merged_data['state'] == 'Oklahoma') | (prop_merged_data['state'] == 'Missouri') |
    (prop_merged_data['state'] == 'Wisconsin') | (prop_merged_data['state'] == 'Minnesota')]
prop_sample

,cofips,county,state,year,type,filings,filing_rate,threatened,threatened_rate,judgements,judgement_rate,year RTW adopted,group,post treatment
3198,17003,Alexander County,Illinois,2000,observed,1,0.09,1,0.09,NaN,NaN,NaN,comparison,0
3199,17003,Alexander County,Illinois,2001,observed,12,1.12,12,1.12,6.0,0.56,NaN,comparison,0
3200,17003,Alexander County,Illinois,2002,observed,19,1.77,19,1.77,NaN,NaN,NaN,comparison,0
3201,17003,Alexander County,Illinois,2003,observed,9,0.84,9,0.84,NaN,NaN,NaN,comparison,0
3202,17003,Alexander County,Illinois,2004,observed,9,0.84,9,0.84,NaN,NaN,NaN,comparison,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21045,55141,Wood County,Wisconsin,2013,observed,200,2.47,192,2.37,154.0,1.90,2015.0,treatment,0
21046,55141,Wood County,Wisconsin,2014,observed,260,3.21,243,3.00,178.0,2.20,2015.0,treatment,0
21047,55141,Wood County,Wisconsin,2015,observed,261,3.22,246,3.04,197.0,2.43,2015.0,treatment,0
21048,55141,Wood County,Wisconsin,2016,observed,281,3.15,263,2.94,191.0,2.14,2015.0,treatment,1


###Add unique identifier cofips_year

In [18]:
#create unique identifier to use when merging Census data

def cofips_year(row):
    year = row["year"]
    cofips = row["cofips"]
    if year < 2005:
        return str(cofips)+"_2000"
    elif (year > 2004) & (year < 2010):
        return str(cofips)+"_2009"
    elif (year > 2009) & (year < 2015):
        return str(cofips)+"_2014"
    elif (year > 2014):
        return str(cofips)+"_2018"
    else:
        return "invalid"

In [19]:
prop_sample["cofips_year"] = prop_sample.apply(cofips_year, axis=1)
prop_sample

<ipython-input-19-0da4600ec22a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_sample["cofips_year"] = prop_sample.apply(cofips_year, axis=1)


,cofips,county,state,year,type,filings,filing_rate,threatened,threatened_rate,judgements,judgement_rate,year RTW adopted,group,post treatment,cofips_year
3198,17003,Alexander County,Illinois,2000,observed,1,0.09,1,0.09,NaN,NaN,NaN,comparison,0,17003_2000
3199,17003,Alexander County,Illinois,2001,observed,12,1.12,12,1.12,6.0,0.56,NaN,comparison,0,17003_2000
3200,17003,Alexander County,Illinois,2002,observed,19,1.77,19,1.77,NaN,NaN,NaN,comparison,0,17003_2000
3201,17003,Alexander County,Illinois,2003,observed,9,0.84,9,0.84,NaN,NaN,NaN,comparison,0,17003_2000
3202,17003,Alexander County,Illinois,2004,observed,9,0.84,9,0.84,NaN,NaN,NaN,comparison,0,17003_2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21045,55141,Wood County,Wisconsin,2013,observed,200,2.47,192,2.37,154.0,1.90,2015.0,treatment,0,55141_2014
21046,55141,Wood County,Wisconsin,2014,observed,260,3.21,243,3.00,178.0,2.20,2015.0,treatment,0,55141_2014
21047,55141,Wood County,Wisconsin,2015,observed,261,3.22,246,3.04,197.0,2.43,2015.0,treatment,0,55141_2018
21048,55141,Wood County,Wisconsin,2016,observed,281,3.15,263,2.94,191.0,2.14,2015.0,treatment,1,55141_2018


##Estimates

This data includes imputed values

###Upload Eviction Lab data

Source: https://data-downloads.evictionlab.org/#estimating-eviction-prevalance-across-us/

File: county_eviction_estimates_2000_2018.csv

In [20]:
link = 'https://drive.google.com/file/d/1slaNDCCmEHZYxv1ieeECcHAPjSFjO_4w/view'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('county_eviction_estimates_2000_2018.csv')

estimates_raw_data = pd.read_csv('county_eviction_estimates_2000_2018.csv')
print('Success!')
estimates_raw_data.head()

Success!


,state,county,FIPS_state,FIPS_county,year,renting_hh,filings_estimate,filings_ci_95_lower,filings_ci_95_upper,ind_filings_court_issued,ind_filings_court_issued_LT,hh_threat_estimate,hh_threat_95_lower,hh_threat_95_upper,ind_hht_observed
0,Alabama,Autauga County,1,1001,2000,3074,109,109,109,1,0.0,106,106,106,1
1,Alabama,Autauga County,1,1001,2001,3264,75,75,75,1,0.0,74,71,75,0
2,Alabama,Autauga County,1,1001,2002,3454,94,94,94,1,0.0,92,88,94,0
3,Alabama,Autauga County,1,1001,2003,3644,100,100,100,1,0.0,98,94,100,0
4,Alabama,Autauga County,1,1001,2004,3834,112,112,112,1,0.0,110,106,112,0


In [21]:
estimates_raw_data["filing_rate"] = (estimates_raw_data["filings_estimate"]/estimates_raw_data["renting_hh"])*100
estimates_raw_data["filing_rate"] = estimates_raw_data["filing_rate"].round(decimals=2)
estimates_raw_data.head()

,state,county,FIPS_state,FIPS_county,year,renting_hh,filings_estimate,filings_ci_95_lower,filings_ci_95_upper,ind_filings_court_issued,ind_filings_court_issued_LT,hh_threat_estimate,hh_threat_95_lower,hh_threat_95_upper,ind_hht_observed,filing_rate
0,Alabama,Autauga County,1,1001,2000,3074,109,109,109,1,0.0,106,106,106,1,3.55
1,Alabama,Autauga County,1,1001,2001,3264,75,75,75,1,0.0,74,71,75,0,2.30
2,Alabama,Autauga County,1,1001,2002,3454,94,94,94,1,0.0,92,88,94,0,2.72
3,Alabama,Autauga County,1,1001,2003,3644,100,100,100,1,0.0,98,94,100,0,2.74
4,Alabama,Autauga County,1,1001,2004,3834,112,112,112,1,0.0,110,106,112,0,2.92


In [22]:
estimates_raw_data = estimates_raw_data.rename(columns={"FIPS_county":"cofips","filings_estimate":"filings"})

###Upload RTW data & merge

In [23]:
#add year that RTW law was passed to eviction dataset

estimates_merged_data = estimates_raw_data.merge(RTW_states, how="left", on="state")

In [24]:
#assign data to treatment and comparison groups

def RTW_year(row):
    year = row["year RTW adopted"]

    if year < 2000:
        return "before 2000"
    elif year >= 2000:
        return "treatment"
    else:
        return "comparison"

estimates_merged_data["group"] = estimates_merged_data.apply(RTW_year, axis=1)

In [25]:
#create pre and post dummy

import numpy as np

year = estimates_merged_data.loc[:,("year")]
state = estimates_merged_data.loc[:,("state")]

conditions = [
    (state == 'Indiana') & (year > 2012),
    (state == 'Kentucky') & (year > 2017),
    (state == 'Michigan') & (year > 2012),
    (state == 'Oklahoma') & (year > 2001),
    (state == 'Wisconsin') & (year > 2015),
]


choices = [1, 1, 1, 1, 1]

estimates_merged_data['post treatment'] = np.select(conditions, choices, default=0)
estimates_merged_data.head()


,state,county,FIPS_state,cofips,year,renting_hh,filings,filings_ci_95_lower,filings_ci_95_upper,ind_filings_court_issued,ind_filings_court_issued_LT,hh_threat_estimate,hh_threat_95_lower,hh_threat_95_upper,ind_hht_observed,filing_rate,year RTW adopted,group,post treatment
0,Alabama,Autauga County,1,1001,2000,3074,109,109,109,1,0.0,106,106,106,1,3.55,1953.0,before 2000,0
1,Alabama,Autauga County,1,1001,2001,3264,75,75,75,1,0.0,74,71,75,0,2.30,1953.0,before 2000,0
2,Alabama,Autauga County,1,1001,2002,3454,94,94,94,1,0.0,92,88,94,0,2.72,1953.0,before 2000,0
3,Alabama,Autauga County,1,1001,2003,3644,100,100,100,1,0.0,98,94,100,0,2.74,1953.0,before 2000,0
4,Alabama,Autauga County,1,1001,2004,3834,112,112,112,1,0.0,110,106,112,0,2.92,1953.0,before 2000,0


###Include only treatment and comparison states

In [26]:
estimates_sample = estimates_merged_data[
    (estimates_merged_data['state'] == 'Indiana') | (estimates_merged_data['state'] == 'Illinois') |
    (estimates_merged_data['state'] == 'Michigan') | (estimates_merged_data['state'] == 'Ohio') |
    (estimates_merged_data['state'] == 'Oklahoma') | (estimates_merged_data['state'] == 'Missouri') |
    (estimates_merged_data['state'] == 'Wisconsin') | (estimates_merged_data['state'] == 'Minnesota')]
estimates_sample

,state,county,FIPS_state,cofips,year,renting_hh,filings,filings_ci_95_lower,filings_ci_95_upper,ind_filings_court_issued,ind_filings_court_issued_LT,hh_threat_estimate,hh_threat_95_lower,hh_threat_95_upper,ind_hht_observed,filing_rate,year RTW adopted,group,post treatment
11305,Illinois,Adams County,17,17001,2000,7060,101,66,147,0,NaN,98,64,143,0,1.43,NaN,comparison,0
11306,Illinois,Adams County,17,17001,2001,7116,91,64,126,0,NaN,89,62,123,0,1.28,NaN,comparison,0
11307,Illinois,Adams County,17,17001,2002,7172,111,74,158,0,NaN,108,72,153,0,1.55,NaN,comparison,0
11308,Illinois,Adams County,17,17001,2003,7227,105,69,150,0,NaN,103,68,146,0,1.45,NaN,comparison,0
11309,Illinois,Adams County,17,17001,2004,7283,115,76,166,0,NaN,113,74,162,0,1.58,NaN,comparison,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59275,Wisconsin,Wood County,55,55141,2014,8936,281,281,281,1,0.0,262,262,262,1,3.14,2015.0,treatment,0
59276,Wisconsin,Wood County,55,55141,2015,9053,261,261,261,1,0.0,246,246,246,1,2.88,2015.0,treatment,0
59277,Wisconsin,Wood County,55,55141,2016,9170,316,316,316,1,0.0,295,295,295,1,3.45,2015.0,treatment,1
59278,Wisconsin,Wood County,55,55141,2017,9287,273,273,273,1,0.0,249,249,249,1,2.94,2015.0,treatment,1


###Add unique identifier cofips_year

In [27]:
#create unique identifier to use when merging Census data

def cofips_year(row):
    year = row["year"]
    cofips = row["cofips"]
    if year < 2005:
        return str(cofips)+"_2000"
    elif (year > 2004) & (year < 2010):
        return str(cofips)+"_2009"
    elif (year > 2009) & (year < 2015):
        return str(cofips)+"_2014"
    elif (year > 2014):
        return str(cofips)+"_2018"
    else:
        return "invalid"

In [28]:
estimates_sample["cofips_year"] = estimates_sample.apply(cofips_year, axis=1)
estimates_sample

<ipython-input-28-7b3fc178c765>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estimates_sample["cofips_year"] = estimates_sample.apply(cofips_year, axis=1)


,state,county,FIPS_state,cofips,year,renting_hh,filings,filings_ci_95_lower,filings_ci_95_upper,ind_filings_court_issued,ind_filings_court_issued_LT,hh_threat_estimate,hh_threat_95_lower,hh_threat_95_upper,ind_hht_observed,filing_rate,year RTW adopted,group,post treatment,cofips_year
11305,Illinois,Adams County,17,17001,2000,7060,101,66,147,0,NaN,98,64,143,0,1.43,NaN,comparison,0,17001_2000
11306,Illinois,Adams County,17,17001,2001,7116,91,64,126,0,NaN,89,62,123,0,1.28,NaN,comparison,0,17001_2000
11307,Illinois,Adams County,17,17001,2002,7172,111,74,158,0,NaN,108,72,153,0,1.55,NaN,comparison,0,17001_2000
11308,Illinois,Adams County,17,17001,2003,7227,105,69,150,0,NaN,103,68,146,0,1.45,NaN,comparison,0,17001_2000
11309,Illinois,Adams County,17,17001,2004,7283,115,76,166,0,NaN,113,74,162,0,1.58,NaN,comparison,0,17001_2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59275,Wisconsin,Wood County,55,55141,2014,8936,281,281,281,1,0.0,262,262,262,1,3.14,2015.0,treatment,0,55141_2014
59276,Wisconsin,Wood County,55,55141,2015,9053,261,261,261,1,0.0,246,246,246,1,2.88,2015.0,treatment,0,55141_2018
59277,Wisconsin,Wood County,55,55141,2016,9170,316,316,316,1,0.0,295,295,295,1,3.45,2015.0,treatment,1,55141_2018
59278,Wisconsin,Wood County,55,55141,2017,9287,273,273,273,1,0.0,249,249,249,1,2.94,2015.0,treatment,1,55141_2018


# Census data/control variables

##State fips codes
* Illinois = 17
* Indiana = 18
* Michigan = 26
* Minnesota = 27
* Missouri = 29
* Ohio = 39
* Oklahoma = 40
* Wisconsin = 55

Variable codes:
https://docs.google.com/spreadsheets/d/16SP7g7x3FgvF3aCcGBoccwhpgZw28GPn3XPtlJ_4_bY/edit#gid=0

##2000 Census

In [29]:
#summary file 3

year = "2000"
dataset = "dec/sf3"
variables = "GEO_ID,NAME,P001001,H006001,H006003,H007001,H007003,H011012,H011014,H012003,H013003,H037003,H063001,H085001,HCT012003,HCT024024,P026001,P026002,P043007,P043014,P043003,P043010"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data = data.json()


In [30]:
#turn json into df

dec2000 = pd.DataFrame(census_data)
dec2000.columns = dec2000.iloc[0]
dec2000 = dec2000[1:]
dec2000.head()

,GEO_ID,NAME,P001001,H006001,H006003,H007001,H007003,H011012,H011014,H012003,...,HCT012003,HCT024024,P026001,P026002,P043007,P043014,P043003,P043010,state,county
1,0500000US17001,"Adams County, Illinois",68277,29386,2526,26860,7049,433,21,44,...,21280,1718,32936,31461,979,766,18325,16797,17,001
2,0500000US17003,"Alexander County, Illinois",9590,4591,783,3808,1065,632,0,2,...,12092,495,3198,2022,206,179,1907,1734,17,003
3,0500000US17005,"Bond County, Illinois",17633,6690,535,6155,1255,51,2,14,...,20494,366,7659,7059,249,184,4441,3713,17,005
4,0500000US17007,"Boone County, Illinois",41786,15414,817,14597,3125,9,9,538,...,30299,615,19755,19383,573,354,11647,9432,17,007
5,0500000US17009,"Brown County, Illinois",6950,2456,348,2108,547,4,0,0,...,20042,131,2503,2479,50,41,1401,1220,17,009


In [31]:
dec2000[
    ["P001001","H006001","H006003","H007001","H007003","H011012","H011014","H012003","H013003","H037003","H063001",
     "HCT012003","HCT024024","P026001","P026002","P043007","P043014","P043003","P043010"]
]=dec2000[
    ["P001001","H006001","H006003","H007001","H007003","H011012","H011014","H012003","H013003","H037003","H063001",
       "HCT012003","HCT024024","P026001","P026002","P043007","P043014","P043003","P043010"]
].astype(int)

In [32]:
dec2000 = dec2000.rename(
    {"NAME" : "name",
     "P001001": "tot_pop",
     "H007001" : "tot_hh",
     "H037003": "med_yr_built",
     "H063001": "med_rent",
     "HCT012003": "med_income",
     "H085001" : "med_home_val"}, axis="columns")

In [33]:
dec2000["cofips"] = dec2000["GEO_ID"].str[9:].astype(int)
dec2000["vac_rate"] = (dec2000["H006003"]/dec2000["H006001"])*100
dec2000["per_rent"] = (dec2000["H007003"]/dec2000["tot_hh"])*100
dec2000["per_blk"] = (dec2000["H011012"]/dec2000["H007003"])*100
dec2000["per_asn"] = (dec2000["H011014"]/dec2000["H007003"])*100
dec2000["per_hsp"] = (dec2000["H012003"]/dec2000["H007003"])*100
dec2000["per_wht"] = (dec2000["H013003"]/dec2000["H007003"])*100
dec2000["per_work_in_state"] = (dec2000["P026002"]/dec2000["P026001"])*100
dec2000["unemp_rate"] = ((dec2000["P043007"]+dec2000["P043014"])/(dec2000["P043003"]+dec2000["P043010"]))*100

In [34]:
#round percents to 1 decimal
dec2000["vac_rate"] = dec2000["vac_rate"].round(decimals=1)
dec2000["per_rent"] = dec2000["per_rent"].round(decimals=1)
dec2000["per_blk"] = dec2000["per_blk"].round(decimals=1)
dec2000["per_asn"] = dec2000["per_asn"].round(decimals=1)
dec2000["per_hsp"] = dec2000["per_hsp"].round(decimals=1)
dec2000["per_wht"] = dec2000["per_wht"].round(decimals=1)
dec2000["per_work_in_state"] = dec2000["per_work_in_state"].round(decimals=1)
dec2000["unemp_rate"] = dec2000["unemp_rate"].round(decimals=1)

In [35]:
#create unique identifier for merging
dec2000["cofips_year"] = dec2000["cofips"].astype(str)+"_2000"

In [36]:
dec2000 = dec2000[["cofips_year","name","tot_pop","tot_hh","vac_rate","per_rent","per_blk","per_asn","per_hsp","per_wht","med_rent","med_income","med_yr_built","med_home_val","per_work_in_state","unemp_rate"]]
dec2000.head()

,cofips_year,name,tot_pop,tot_hh,vac_rate,per_rent,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
1,17001_2000,"Adams County, Illinois",68277,26860,8.6,26.2,6.1,0.3,0.6,91.7,402,21280,1949,73100,95.5,5.0
2,17003_2000,"Alexander County, Illinois",9590,3808,17.1,28.0,59.3,0.0,0.2,39.8,265,12092,1960,32600,63.2,10.6
3,17005_2000,"Bond County, Illinois",17633,6155,8.0,20.4,4.1,0.2,1.1,93.7,385,20494,1963,68200,92.2,5.3
4,17007_2000,"Boone County, Illinois",41786,14597,5.3,21.4,0.3,0.3,17.2,80.9,531,30299,1962,120900,98.1,4.4
5,17009_2000,"Brown County, Illinois",6950,2108,14.2,25.9,0.7,0.0,0.0,99.3,316,20042,1958,50800,99.0,3.5


##2009 ACS

In [37]:
#data profile

year = "2009"
dataset = "acs/acs5/profile"
variables = "GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0046E,DP04_0046PE,DP04_0088E,DP04_0132E,DP05_0001E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data = data.json()

In [38]:
#turn json into df

acs2009dp = pd.DataFrame(census_data)
acs2009dp.columns = acs2009dp.iloc[0]
acs2009dp = acs2009dp[1:]
acs2009dp.head()

,GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0046E,DP04_0046PE,DP04_0088E,DP04_0132E,DP05_0001E,state,county
1,0500000US17001,"Adams County, Illinois",35247,1876,30383,27579,9.2,7429,26.9,96600,533,66892,17,001
2,0500000US17003,"Alexander County, Illinois",3554,348,4643,3603,22.4,1022,28.4,54900,421,8254,17,003
3,0500000US17005,"Bond County, Illinois",9300,1016,7193,6310,12.3,1184,18.8,103000,623,18173,17,005
4,0500000US17007,"Boone County, Illinois",25744,2407,18507,16899,8.7,3330,19.7,166700,690,52400,17,007
5,0500000US17009,"Brown County, Illinois",2333,98,2468,2148,13.0,603,28.1,71800,443,6593,17,009


In [39]:
#detailed table

year = "2009"
dataset = "acs/acs5"
variables = "GEO_ID,B08007_001E,B08007_002E,B25003_003E,B25003B_003E,B25003D_003E,B25003H_003E,B25003I_003E,B25037_003E,B25119_003E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data3 = data.json()


In [40]:
#turn json into df

acs2009dt = pd.DataFrame(census_data3)
acs2009dt.columns = acs2009dt.iloc[0]
acs2009dt = acs2009dt[1:]
acs2009dt.head()

,GEO_ID,B08007_001E,B08007_002E,B25003_003E,B25003B_003E,B25003D_003E,B25003H_003E,B25003I_003E,B25037_003E,B25119_003E,state,county
1,0500000US18049,8776,8669,2136,30,0,1998,90,1973,24836,18,049
2,0500000US18045,7768,6721,1509,0,0,1465,44,1964,26840,18,045
3,0500000US18047,10317,7900,1552,0,19,1520,3,1971,26406,18,047
4,0500000US18051,15376,14887,2891,45,45,2718,55,1973,25795,18,051
5,0500000US18053,28813,28512,8019,948,36,6596,304,1970,21774,18,053


In [41]:
acs2009 = acs2009dp.merge(acs2009dt, how="left", left_on="GEO_ID", right_on="GEO_ID")
acs2009.head()

,GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0046E,DP04_0046PE,DP04_0088E,...,B08007_002E,B25003_003E,B25003B_003E,B25003D_003E,B25003H_003E,B25003I_003E,B25037_003E,B25119_003E,state_y,county_y
0,0500000US17001,"Adams County, Illinois",35247,1876,30383,27579,9.2,7429,26.9,96600,...,31004,7429,469,70,6706,65,1958,23424,17,001
1,0500000US17003,"Alexander County, Illinois",3554,348,4643,3603,22.4,1022,28.4,54900,...,1731,1022,710,0,310,0,1959,12355,17,003
2,0500000US17005,"Bond County, Illinois",9300,1016,7193,6310,12.3,1184,18.8,103000,...,7542,1184,125,0,1044,15,1961,24250,17,005
3,0500000US17007,"Boone County, Illinois",25744,2407,18507,16899,8.7,3330,19.7,166700,...,22222,3330,244,18,2135,907,1964,26998,17,007
4,0500000US17009,"Brown County, Illinois",2333,98,2468,2148,13.0,603,28.1,71800,...,2145,603,2,0,492,0,1955,21208,17,009


In [42]:
acs2009[
    ["B08007_001E","B08007_002E","B25003_003E","B25003B_003E","B25003D_003E","B25003H_003E","B25003I_003E","DP03_0002E","DP03_0005E"]
] =acs2009[
    ["B08007_001E","B08007_002E","B25003_003E","B25003B_003E","B25003D_003E","B25003H_003E","B25003I_003E","DP03_0002E","DP03_0005E"]
].astype(int)

In [43]:
acs2009 = acs2009.rename(
    {"NAME" : "name",
     "DP05_0001E": "tot_pop",
     "DP04_0002E" : "tot_hh",
     "DP04_0003PE" : "vac_rate",
     "DP04_0046PE" : "per_rent",
     "B25037_003E" : "med_yr_built",
     "DP04_0132E": "med_rent",
     "B25119_003E": "med_income",
     "DP04_0088E" : "med_home_val"}, axis="columns")

In [44]:
acs2009["cofips"] = acs2009["GEO_ID"].str[9:].astype(int)
acs2009["per_blk"] = (acs2009["B25003B_003E"]/acs2009["B25003_003E"])*100
acs2009["per_asn"] = (acs2009["B25003D_003E"]/acs2009["B25003_003E"])*100
acs2009["per_hsp"] = (acs2009["B25003I_003E"]/acs2009["B25003_003E"])*100
acs2009["per_wht"] = (acs2009["B25003H_003E"]/acs2009["B25003_003E"])*100
acs2009["per_work_in_state"] = (acs2009["B08007_002E"]/acs2009["B08007_001E"])*100
acs2009["unemp_rate"] = (acs2009["DP03_0005E"]/acs2009["DP03_0002E"])*100

In [45]:
#round percents to 1 decimal
acs2009["per_blk"] = acs2009["per_blk"].round(decimals=1)
acs2009["per_asn"] = acs2009["per_asn"].round(decimals=1)
acs2009["per_hsp"] = acs2009["per_hsp"].round(decimals=1)
acs2009["per_wht"] = acs2009["per_wht"].round(decimals=1)
acs2009["per_work_in_state"] = acs2009["per_work_in_state"].round(decimals=1)
acs2009["unemp_rate"] = acs2009["unemp_rate"].round(decimals=1)

In [46]:
#create unique identifier for merging
acs2009["cofips_year"] = acs2009["cofips"].astype(str)+"_2009"

In [47]:
acs2009 = acs2009[["cofips_year","name","tot_pop","tot_hh","vac_rate","per_rent","per_blk","per_asn","per_hsp","per_wht","med_rent","med_income","med_yr_built","med_home_val","per_work_in_state","unemp_rate"]]
acs2009.head()

,cofips_year,name,tot_pop,tot_hh,vac_rate,per_rent,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
0,17001_2009,"Adams County, Illinois",66892,27579,9.2,26.9,6.3,0.9,0.9,90.3,533,23424,1958,96600,95.0,5.3
1,17003_2009,"Alexander County, Illinois",8254,3603,22.4,28.4,69.5,0.0,0.0,30.3,421,12355,1959,54900,58.2,9.8
2,17005_2009,"Bond County, Illinois",18173,6310,12.3,18.8,10.6,0.0,1.3,88.2,623,24250,1961,103000,91.7,10.9
3,17007_2009,"Boone County, Illinois",52400,16899,8.7,19.7,7.3,0.5,27.2,64.1,690,26998,1964,166700,97.4,9.3
4,17009_2009,"Brown County, Illinois",6593,2148,13.0,28.1,0.3,0.0,0.0,81.6,443,21208,1955,71800,99.2,4.2


##2014 ACS

In [48]:
#data profile

year = "2014"
dataset = "acs/acs5/profile"
variables = "GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0046E,DP04_0046PE,DP04_0088E,DP04_0132E,DP05_0001E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data = data.json()

In [49]:
#turn json into df

acs2014dp = pd.DataFrame(census_data)
acs2014dp.columns = acs2014dp.iloc[0]
acs2014dp = acs2014dp[1:]
acs2014dp.head()

,GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0046E,DP04_0046PE,DP04_0088E,DP04_0132E,DP05_0001E,state,county
1,0500000US29205,"Shelby County, Missouri",3081,217,3190,2539,20.4,682,26.9,72000,443,6218,29,205
2,0500000US29225,"Webster County, Missouri",15935,1275,14451,12681,12.2,3283,25.9,117000,590,36461,29,225
3,0500000US29005,"Atchison County, Missouri",2807,137,2973,2465,17.1,765,31.0,79900,553,5520,29,005
4,0500000US29013,"Bates County, Missouri",7916,702,7809,6784,13.1,1898,28.0,100200,600,16780,29,013
5,0500000US29015,"Benton County, Missouri",7233,611,14079,8273,41.2,1322,16.0,105400,589,18961,29,015


In [50]:
#subject table

year = "2014"
dataset = "acs/acs5/subject"
variables = "GEO_ID,S0801_C01_014E,S2502_C03_003E,S2502_C03_005E,S2502_C03_009E,S2502_C03_010E,S2503_C03_013E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data2 = data.json()


In [51]:
#turn json into df

acs2014sub = pd.DataFrame(census_data2)
acs2014sub.columns = acs2014sub.iloc[0]
acs2014sub = acs2014sub[1:]
acs2014sub.head()

,GEO_ID,S0801_C01_014E,S2502_C03_003E,S2502_C03_005E,S2502_C03_009E,S2502_C03_010E,S2503_C03_013E,state,county
1,0500000US29205,97.6,0.0,0.0,1.9,97.4,23409,29,205
2,0500000US29225,98.3,0.2,0.8,3.3,92.2,25154,29,225
3,0500000US29005,82.2,0.0,0.5,1.0,95.3,35048,29,005
4,0500000US29013,86.2,0.9,0.0,1.0,95.7,26610,29,013
5,0500000US29015,97.4,0.5,0.0,0.7,98.8,17451,29,015


In [52]:
#detailed table

year = "2014"
dataset = "acs/acs5"
variables = "GEO_ID,B25037_003E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data3 = data.json()


In [53]:
#turn json into df

acs2014dt = pd.DataFrame(census_data3)
acs2014dt.columns = acs2014dt.iloc[0]
acs2014dt = acs2014dt[1:]
acs2014dt.head()

,GEO_ID,B25037_003E,state,county
1,0500000US29205,1971,29,205
2,0500000US29225,1983,29,225
3,0500000US29005,1961,29,005
4,0500000US29013,1967,29,013
5,0500000US29015,1977,29,015


In [54]:
merge1 = acs2014dp.merge(acs2014sub, how="left", left_on="GEO_ID", right_on="GEO_ID")
acs2014 = merge1.merge(acs2014dt, how="left", left_on="GEO_ID", right_on="GEO_ID")
acs2014.head()

,GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0046E,DP04_0046PE,DP04_0088E,...,S2502_C03_003E,S2502_C03_005E,S2502_C03_009E,S2502_C03_010E,S2503_C03_013E,state_y,county_y,B25037_003E,state,county
0,0500000US29205,"Shelby County, Missouri",3081,217,3190,2539,20.4,682,26.9,72000,...,0.0,0.0,1.9,97.4,23409,29,205,1971,29,205
1,0500000US29225,"Webster County, Missouri",15935,1275,14451,12681,12.2,3283,25.9,117000,...,0.2,0.8,3.3,92.2,25154,29,225,1983,29,225
2,0500000US29005,"Atchison County, Missouri",2807,137,2973,2465,17.1,765,31.0,79900,...,0.0,0.5,1.0,95.3,35048,29,005,1961,29,005
3,0500000US29013,"Bates County, Missouri",7916,702,7809,6784,13.1,1898,28.0,100200,...,0.9,0.0,1.0,95.7,26610,29,013,1967,29,013
4,0500000US29015,"Benton County, Missouri",7233,611,14079,8273,41.2,1322,16.0,105400,...,0.5,0.0,0.7,98.8,17451,29,015,1977,29,015


In [55]:
acs2014[["DP03_0002E","DP03_0005E"]]=acs2014[["DP03_0002E","DP03_0005E"]].astype(int)

In [56]:
acs2014 = acs2014.rename(
    {"NAME" : "name",
     "DP05_0001E": "tot_pop",
     "DP04_0002E" : "tot_hh",
     "DP04_0003PE" : "vac_rate",
     "DP04_0046PE" : "per_rent",
     "B25037_003E" : "med_yr_built",
     "DP04_0132E": "med_rent",
     "S0801_C01_014E":"per_work_in_state",
     "S2502_C03_003E" : "per_blk",
     "S2502_C03_005E" : "per_asn",
     "S2502_C03_009E" : "per_hsp",
     "S2502_C03_010E": "per_wht",
     "S2503_C03_013E": "med_income",
     "DP04_0088E" : "med_home_val"}, axis="columns")

In [57]:
acs2014["cofips"] = acs2014["GEO_ID"].str[9:].astype(int)
acs2014["unemp_rate"] = (acs2014["DP03_0005E"]/acs2014["DP03_0002E"])*100
acs2014["unemp_rate"] = acs2014["unemp_rate"].round(decimals=1)

In [58]:
#create unique identifier for merging
acs2014["cofips_year"] = acs2014["cofips"].astype(str)+"_2014"

In [59]:
acs2014 = acs2014[["cofips_year","name","tot_pop","tot_hh","vac_rate","per_rent","per_blk","per_asn","per_hsp","per_wht","med_rent","med_income","med_yr_built","med_home_val","per_work_in_state","unemp_rate"]]
acs2014.head()

,cofips_year,name,tot_pop,tot_hh,vac_rate,per_rent,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
0,29205_2014,"Shelby County, Missouri",6218,2539,20.4,26.9,0.0,0.0,1.9,97.4,443,23409,1971,72000,97.6,7.0
1,29225_2014,"Webster County, Missouri",36461,12681,12.2,25.9,0.2,0.8,3.3,92.2,590,25154,1983,117000,98.3,8.0
2,29005_2014,"Atchison County, Missouri",5520,2465,17.1,31.0,0.0,0.5,1.0,95.3,553,35048,1961,79900,82.2,4.9
3,29013_2014,"Bates County, Missouri",16780,6784,13.1,28.0,0.9,0.0,1.0,95.7,600,26610,1967,100200,86.2,8.9
4,29015_2014,"Benton County, Missouri",18961,8273,41.2,16.0,0.5,0.0,0.7,98.8,589,17451,1977,105400,97.4,8.4


##2018 ACS

In [60]:
#data profile

year = "2018"
dataset = "acs/acs5/profile"
variables = "GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0047PE,DP04_0089E,DP04_0134E,DP05_0001E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data4 = data.json()

In [61]:
#turn json into df

acs2018dp = pd.DataFrame(census_data4)
acs2018dp.columns = acs2018dp.iloc[0]
acs2018dp = acs2018dp[1:]
acs2018dp.head()

,GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0047PE,DP04_0089E,DP04_0134E,DP05_0001E,state,county
1,0500000US29029,"Camden County, Missouri",19528,1262,41714,16096,61.4,19.8,183500,757,45096,29,029
2,0500000US29099,"Jefferson County, Missouri",117120,6624,90489,84393,6.7,20.3,158100,848,223302,29,099
3,0500000US29143,"New Madrid County, Missouri",7251,629,8618,7312,15.2,38.2,75500,620,17811,29,143
4,0500000US29173,"Ralls County, Missouri",5245,234,5191,4004,22.9,16.5,136100,774,10217,29,173
5,0500000US29129,"Mercer County, Missouri",1705,43,2130,1326,37.7,23.4,87500,510,3664,29,129


In [62]:
#subject table

year = "2018"
dataset = "acs/acs5/subject"
variables = "GEO_ID,S0801_C01_014E,S2502_C06_003E,S2502_C06_005E,S2502_C06_009E,S2502_C06_010E,S2503_C05_013E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data5 = data.json()

In [63]:
#turn json into df

acs2018sub = pd.DataFrame(census_data5)
acs2018sub.columns = acs2018sub.iloc[0]
acs2018sub = acs2018sub[1:]
acs2018sub.head()

,GEO_ID,S0801_C01_014E,S2502_C06_003E,S2502_C06_005E,S2502_C06_009E,S2502_C06_010E,S2503_C05_013E,state,county
1,0500000US29029,98.0,0.4,0.0,2.0,93.1,30749,29,029
2,0500000US29099,97.8,2.1,0.2,3.0,92.3,37856,29,099
3,0500000US29143,96.0,27.4,0.0,1.2,69.5,23963,29,143
4,0500000US29173,92.8,2.0,0.0,0.0,97.9,35833,29,173
5,0500000US29065,99.6,0.0,0.0,2.3,92.9,20000,29,065


In [64]:
#detailed table

year = "2018"
dataset = "acs/acs5"
variables = "GEO_ID,B25037_003E"
geographies = "for=county:*&in=state:17,18,26,27,29,39,40,55"
key = "a4c07c42ce6f2692353811e122950b9a11b227a2"
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables}&{geographies}&={key}"

data = requests.get(url)
census_data6 = data.json()

In [65]:
#turn json into df

acs2018dt = pd.DataFrame(census_data6)
acs2018dt.columns = acs2018dt.iloc[0]
acs2018dt = acs2018dt[1:]
acs2018dt.head()

,GEO_ID,B25037_003E,state,county
1,0500000US26033,1969,26,033
2,0500000US26115,1973,26,115
3,0500000US26125,1974,26,125
4,0500000US26059,1972,26,059
5,0500000US26121,1965,26,121


In [66]:
merge2 = acs2018dp.merge(acs2018sub, how="left", left_on="GEO_ID", right_on="GEO_ID")
acs2018 = merge2.merge(acs2018dt, how="left", left_on="GEO_ID", right_on="GEO_ID")
acs2018.head()

,GEO_ID,NAME,DP03_0002E,DP03_0005E,DP04_0001E,DP04_0002E,DP04_0003PE,DP04_0047PE,DP04_0089E,DP04_0134E,...,S2502_C06_003E,S2502_C06_005E,S2502_C06_009E,S2502_C06_010E,S2503_C05_013E,state_y,county_y,B25037_003E,state,county
0,0500000US29029,"Camden County, Missouri",19528,1262,41714,16096,61.4,19.8,183500,757,...,0.4,0.0,2.0,93.1,30749,29,029,1988,29,029
1,0500000US29099,"Jefferson County, Missouri",117120,6624,90489,84393,6.7,20.3,158100,848,...,2.1,0.2,3.0,92.3,37856,29,099,1981,29,099
2,0500000US29143,"New Madrid County, Missouri",7251,629,8618,7312,15.2,38.2,75500,620,...,27.4,0.0,1.2,69.5,23963,29,143,1976,29,143
3,0500000US29173,"Ralls County, Missouri",5245,234,5191,4004,22.9,16.5,136100,774,...,2.0,0.0,0.0,97.9,35833,29,173,1976,29,173
4,0500000US29129,"Mercer County, Missouri",1705,43,2130,1326,37.7,23.4,87500,510,...,0.0,0.0,2.3,97.1,30658,29,129,1971,29,129


In [67]:
acs2018 = acs2018.rename(
    {"NAME" : "name",
     "DP05_0001E": "tot_pop",
     "DP04_0002E" : "tot_hh",
     "DP04_0003PE" : "vac_rate",
     "DP04_0047PE" : "per_rent",
     "B25037_003E" : "med_yr_built",
     "DP04_0134E": "med_rent",
     "S0801_C01_014E":"per_work_in_state",
     "S2502_C06_003E" : "per_blk",
     "S2502_C06_005E" : "per_asn",
     "S2502_C06_009E" : "per_hsp",
     "S2502_C06_010E": "per_wht",
     "S2503_C05_013E": "med_income",
     "DP04_0089E" : "med_home_val"}, axis="columns")

In [68]:
acs2018[["DP03_0002E","DP03_0005E"]]=acs2018[["DP03_0002E","DP03_0005E"]].astype(int)

In [69]:
acs2018["cofips"] = acs2018["GEO_ID"].str[9:].astype(int)
acs2018["unemp_rate"] = (acs2018["DP03_0005E"]/acs2018["DP03_0002E"])*100
acs2018["unemp_rate"] = acs2018["unemp_rate"].round(decimals=1)

In [70]:
#create unique identifier for merging
acs2018["cofips_year"] = acs2018["cofips"].astype(str)+"_2018"

In [71]:
acs2018=acs2018[["cofips_year","name","tot_pop","tot_hh","vac_rate","per_rent","per_blk","per_asn","per_hsp","per_wht","med_rent","med_income","med_yr_built","med_home_val","per_work_in_state","unemp_rate"]]
acs2018.head()

,cofips_year,name,tot_pop,tot_hh,vac_rate,per_rent,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
0,29029_2018,"Camden County, Missouri",45096,16096,61.4,19.8,0.4,0.0,2.0,93.1,757,30749,1988,183500,98.0,6.5
1,29099_2018,"Jefferson County, Missouri",223302,84393,6.7,20.3,2.1,0.2,3.0,92.3,848,37856,1981,158100,97.8,5.7
2,29143_2018,"New Madrid County, Missouri",17811,7312,15.2,38.2,27.4,0.0,1.2,69.5,620,23963,1976,75500,96.0,8.7
3,29173_2018,"Ralls County, Missouri",10217,4004,22.9,16.5,2.0,0.0,0.0,97.9,774,35833,1976,136100,92.8,4.5
4,29129_2018,"Mercer County, Missouri",3664,1326,37.7,23.4,0.0,0.0,2.3,97.1,510,30658,1971,87500,92.6,2.5


##Concatenate census dataframes

In [72]:
all_census_data = [dec2000,acs2009,acs2014,acs2018]
controls = pd.concat(all_census_data)
controls

,cofips_year,name,tot_pop,tot_hh,vac_rate,per_rent,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
1,17001_2000,"Adams County, Illinois",68277,26860,8.6,26.2,6.1,0.3,0.6,91.7,402,21280,1949,73100,95.5,5.0
2,17003_2000,"Alexander County, Illinois",9590,3808,17.1,28.0,59.3,0.0,0.2,39.8,265,12092,1960,32600,63.2,10.6
3,17005_2000,"Bond County, Illinois",17633,6155,8.0,20.4,4.1,0.2,1.1,93.7,385,20494,1963,68200,92.2,5.3
4,17007_2000,"Boone County, Illinois",41786,14597,5.3,21.4,0.3,0.3,17.2,80.9,531,30299,1962,120900,98.1,4.4
5,17009_2000,"Brown County, Illinois",6950,2108,14.2,25.9,0.7,0.0,0.0,99.3,316,20042,1958,50800,99.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,55141_2018,"Wood County, Wisconsin",73274,32245,7.5,27.4,0.8,2.6,3.0,90.5,696,32473,1974,129900,99.6,3.8
712,55003_2018,"Ashland County, Wisconsin",15712,6546,32.2,31.7,0.0,1.0,3.1,76.8,636,25393,1962,110100,98.7,5.9
713,55037_2018,"Florence County, Wisconsin",4337,2010,58.0,15.2,0.0,0.0,0.0,100.0,455,30357,1973,130800,45.7,6.1
714,55055_2018,"Jefferson County, Wisconsin",84652,32866,7.9,29.9,0.8,1.1,9.2,87.3,854,38339,1973,184300,99.2,3.7


In [73]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(controls)

Your selected dataframe has 16 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values
0,,


In [74]:
#download csv file

#from google.colab import files
#controls.to_csv('controls.csv',index=False)
#files.download('controls.csv')

#Merge eviction/RTW data with controls

##Court-issued

In [75]:
court_sample = court_sample.merge(controls, how="left", left_on="cofips_year", right_on="cofips_year",validate="m:1")
court_sample.head()

,state,county,fips_state,cofips,year,renting_hh,filings,ind_filings_court_issued_lt,threatened,filing_rate,...,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
0,Illinois,Cook County,17,17031,2001,830604,37452,0,NaN,4.51,...,33.2,5.3,17.6,41.9,648,30634,1955,154300,98.9,7.5
1,Illinois,Cook County,17,17031,2002,829703,35331,0,NaN,4.26,...,33.2,5.3,17.6,41.9,648,30634,1955,154300,98.9,7.5
2,Illinois,Cook County,17,17031,2003,828803,32975,0,NaN,3.98,...,33.2,5.3,17.6,41.9,648,30634,1955,154300,98.9,7.5
3,Illinois,Cook County,17,17031,2004,827902,31937,0,NaN,3.86,...,33.2,5.3,17.6,41.9,648,30634,1955,154300,98.9,7.5
4,Illinois,Cook County,17,17031,2005,827002,30278,0,NaN,3.66,...,35.8,5.5,19.1,38.5,882,32372,1953,267200,98.8,9.3


In [76]:
court_sample = court_sample[["cofips","cofips_year","county","state","year","filings","filing_rate","year RTW adopted","group","post treatment","tot_pop","tot_hh","vac_rate","per_rent","per_asn","per_blk","per_hsp","per_wht","med_rent","med_income","med_yr_built","med_home_val","per_work_in_state","unemp_rate"]]

In [77]:
missing_values_table(court_sample)

Your selected dataframe has 24 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
year RTW adopted,4099,65.2


In [78]:
#download csv file

from google.colab import files
court_sample.to_csv('court_sample.csv',index=False)
files.download('court_sample.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Proprietary

In [79]:
prop_sample = prop_sample.merge(controls, how="left", left_on="cofips_year", right_on="cofips_year",validate="m:1")
prop_sample.head()

,cofips,county,state,year,type,filings,filing_rate,threatened,threatened_rate,judgements,...,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
0,17003,Alexander County,Illinois,2000,observed,1,0.09,1,0.09,NaN,...,59.3,0.0,0.2,39.8,265,12092,1960,32600,63.2,10.6
1,17003,Alexander County,Illinois,2001,observed,12,1.12,12,1.12,6.0,...,59.3,0.0,0.2,39.8,265,12092,1960,32600,63.2,10.6
2,17003,Alexander County,Illinois,2002,observed,19,1.77,19,1.77,NaN,...,59.3,0.0,0.2,39.8,265,12092,1960,32600,63.2,10.6
3,17003,Alexander County,Illinois,2003,observed,9,0.84,9,0.84,NaN,...,59.3,0.0,0.2,39.8,265,12092,1960,32600,63.2,10.6
4,17003,Alexander County,Illinois,2004,observed,9,0.84,9,0.84,NaN,...,59.3,0.0,0.2,39.8,265,12092,1960,32600,63.2,10.6


In [80]:
prop_sample = prop_sample[["cofips","cofips_year","county","state","year","filings","filing_rate","year RTW adopted","group","post treatment","tot_pop","tot_hh","vac_rate","per_rent","per_asn","per_blk","per_hsp","per_wht","med_rent","med_income","med_yr_built","med_home_val","per_work_in_state","unemp_rate"]]

In [81]:
missing_values_table(prop_sample)

Your selected dataframe has 24 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
year RTW adopted,4229,63.4


In [82]:
#download csv file

from google.colab import files
prop_sample.to_csv('prop_sample.csv',index=False)
files.download('prop_sample.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Estimates

In [83]:
estimates_sample = estimates_sample.merge(controls, how="left", left_on="cofips_year", right_on="cofips_year",validate="m:1")
estimates_sample.head()

,state,county,FIPS_state,cofips,year,renting_hh,filings,filings_ci_95_lower,filings_ci_95_upper,ind_filings_court_issued,...,per_blk,per_asn,per_hsp,per_wht,med_rent,med_income,med_yr_built,med_home_val,per_work_in_state,unemp_rate
0,Illinois,Adams County,17,17001,2000,7060,101,66,147,0,...,6.1,0.3,0.6,91.7,402,21280,1949,73100,95.5,5.0
1,Illinois,Adams County,17,17001,2001,7116,91,64,126,0,...,6.1,0.3,0.6,91.7,402,21280,1949,73100,95.5,5.0
2,Illinois,Adams County,17,17001,2002,7172,111,74,158,0,...,6.1,0.3,0.6,91.7,402,21280,1949,73100,95.5,5.0
3,Illinois,Adams County,17,17001,2003,7227,105,69,150,0,...,6.1,0.3,0.6,91.7,402,21280,1949,73100,95.5,5.0
4,Illinois,Adams County,17,17001,2004,7283,115,76,166,0,...,6.1,0.3,0.6,91.7,402,21280,1949,73100,95.5,5.0


In [84]:
estimates_sample = estimates_sample[["cofips","cofips_year","county","state","year","filings","filing_rate","filings_ci_95_lower","filings_ci_95_upper","renting_hh","year RTW adopted","group","post treatment","tot_pop","tot_hh","vac_rate","per_rent","per_asn","per_blk","per_hsp","per_wht","med_rent","med_income","med_yr_built","med_home_val","per_work_in_state","unemp_rate"]]

In [85]:
missing_values_table(prop_sample)

Your selected dataframe has 24 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
year RTW adopted,4229,63.4


In [86]:
#download csv file

from google.colab import files
estimates_sample.to_csv('estimates_sample.csv',index=False)
files.download('estimates_sample.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>